In [27]:
import pandas as pd
import joblib
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [34]:
model_reg = joblib.load('all_features_xgb.joblib')

In [67]:
data = pd.read_csv('data/all_features.csv', index_col=[0,1])

In [68]:
data[["statistical_parity", "equal_opportunity", "average_odds"]] = data[
    ["statistical_parity", "equal_opportunity", "average_odds"]
].abs()

In [69]:
train, test = train_test_split(data, test_size=0.2, shuffle=False)

In [43]:
model_reg.fit(train.drop(columns=['statistical_parity', 'equal_opportunity', 'average_odds']).values, train[['statistical_parity', 'equal_opportunity', 'average_odds']].values)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [45]:
predictions = model_reg.predict(test.drop(columns=['statistical_parity', 'equal_opportunity', 'average_odds']).values)

In [46]:
true_val = test[['statistical_parity', 'equal_opportunity', 'average_odds']]

In [47]:
true_val.loc[:,'stat_par_pred'] = predictions[:, 0]
true_val.loc[:,'eq_opp_pred'] = predictions[:, 1]
true_val.loc[:,'avg_odds_pred'] = predictions[:, 2]

c:\Users\giord\miniconda3\envs\bias\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [48]:
true_val

,,statistical_parity,equal_opportunity,average_odds,stat_par_pred,eq_opp_pred,avg_odds_pred
variable,data,,,,,,
age_65,health,0.002187,0.000467,0.005576,0.005184,0.026977,0.017148
age_75,health,0.001317,0.080599,0.015067,0.002520,0.071845,0.000773
age_85,health,0.002351,0.133723,0.000595,-0.004387,0.122724,0.023963
age_MISS,health,0.013354,0.169971,0.012140,0.038404,0.156345,0.043793
sexMALE,health,0.000428,0.030862,0.004383,-0.003729,0.025852,-0.004250
...,...,...,...,...,...,...,...
type,wine,0.159525,0.014760,0.299304,0.170590,0.030387,0.062462
alcohol,wine,0.640531,0.015267,1.253022,0.344491,0.115946,0.321133
type,wine,0.235537,0.039945,0.467955,0.230214,0.076621,0.110527


In [49]:
r2_score(true_val['statistical_parity'], true_val['stat_par_pred'])

0.7044014037149324

In [50]:
r2_score(true_val['equal_opportunity'], true_val['eq_opp_pred'])

0.03385115544053474

In [51]:
r2_score(true_val['average_odds'], true_val['avg_odds_pred'])

0.17737567676010968

## Change the problem into classification with threshold

In [64]:
model = XGBClassifier()

In [70]:
train.loc[train["statistical_parity"] > 0.2, "statistical_parity"] = 1
train.loc[train["statistical_parity"] != 1, "statistical_parity"] = 0
train.loc[train["equal_opportunity"] > 0.2, "equal_opportunity"] = 1
train.loc[train["equal_opportunity"] != 1, "equal_opportunity"] = 0
train.loc[train["average_odds"] > 0.2, "average_odds"] = 1
train.loc[train["average_odds"] != 1, "average_odds"] = 0

In [71]:
test.loc[test["statistical_parity"] > 0.2, "statistical_parity"] = 1
test.loc[test["statistical_parity"] != 1, "statistical_parity"] = 0
test.loc[test["equal_opportunity"] > 0.2, "equal_opportunity"] = 1
test.loc[test["equal_opportunity"] != 1, "equal_opportunity"] = 0
test.loc[test["average_odds"] > 0.2, "average_odds"] = 1
test.loc[test["average_odds"] != 1, "average_odds"] = 0

In [74]:
model.fit(train.drop(columns=['statistical_parity', 'equal_opportunity', 'average_odds']).values, train[['statistical_parity', 'equal_opportunity', 'average_odds']].values)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [75]:
predictions = model.predict(
    test.drop(
        columns=["statistical_parity", "equal_opportunity", "average_odds"]
    ).values
)

In [76]:
true_val = test[["statistical_parity", "equal_opportunity", "average_odds"]]
true_val.loc[:, "stat_par_pred"] = predictions[:, 0]
true_val.loc[:, "eq_opp_pred"] = predictions[:, 1]
true_val.loc[:, "avg_odds_pred"] = predictions[:, 2]

c:\Users\giord\miniconda3\envs\bias\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [78]:
from sklearn.metrics import accuracy_score

accuracy_score(true_val["statistical_parity"], true_val["stat_par_pred"])

0.9296551724137931

In [79]:
accuracy_score(true_val["equal_opportunity"], true_val["eq_opp_pred"])

0.8303448275862069

In [80]:
accuracy_score(true_val["average_odds"], true_val["avg_odds_pred"])

0.593103448275862